In [ ]:
import torch
from torch_geometric.datasets import OPFDataset
import pandas
import scipy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [2]:
num_samples = 10000
num_buses = 118
num_loads = 99
num_gens = 54

In [3]:
## Load pfdelta data
path_to_data = "/Users/tamaragovindasamy/Desktop/gridfm-datakit/data/pfdelta/30000opfv2dataset_case118_ieee.mat"
pfdelta_data = scipy.io.loadmat(path_to_data)
baseMVA = pfdelta_data["baseMVA"]
print(baseMVA.dtype)
pd_ = pfdelta_data["bus"][:, 2]
qd_ = pfdelta_data["bus"][:, 3]
load_bus_mask = (pd_ > 0) | (qd_ > 0)
gen_bus = (
    torch.tensor(pfdelta_data["gen"][:, 0], dtype=torch.long) - 1
)  # change index of bus back to
gen_bus_expanded = gen_bus.expand(num_samples, -1)
spv_bus_mask = torch.tensor(pfdelta_data["bus"][:, 1] != 1, dtype=torch.bool)

# Get variables (only first 10000 samples)
pd = np.real(pfdelta_data["Dem"][load_bus_mask, :10000].T) / 100
qd = np.imag(pfdelta_data["Dem"][load_bus_mask, :10000].T) / 100
pg = torch.tensor(np.real(pfdelta_data["Gen"][:10000, :].T) / 100)
qg = torch.tensor(np.imag(pfdelta_data["Gen"][:10000, :].T) / 100)
vm = np.abs(pfdelta_data["Vol"][:10000, :].T)

pg_full = torch.zeros((num_samples, num_buses), dtype=pg.dtype)
qg_full = torch.zeros((num_samples, num_buses), dtype=qg.dtype)
pg_full.scatter_add_(1, gen_bus_expanded, pg)
qg_full.scatter_add_(1, gen_bus_expanded, qg)
pg = pg_full[:, spv_bus_mask]
qg = qg_full[:, spv_bus_mask]

perturbpf = {"pd": pd, "qd": qd, "pg": pg, "qg": qg, "vm": vm}

uint8


In [ ]:
gen_bus

In [4]:
# Load OPFData
root = "opfdata"
dataset = OPFDataset(
    root=root,
    split="train",
    case_name="pglib_opf_case118_ieee",
    num_groups=1,
    topological_perturbations=True,
    force_reload=False,
)

pg_full = torch.zeros((num_samples, num_buses))
qg_full = torch.zeros((num_samples, num_buses))
pd = torch.zeros((num_samples, num_loads))
qd = torch.zeros((num_samples, num_loads))
vm = torch.zeros((num_samples, num_buses))

for i in range(10000):
    pg = dataset[i]["generator"]["y"][:, 0]
    qg = dataset[i]["generator"]["y"][:, 1]
    gen_bus = dataset[i]["generator", "generator_link", "bus"]["edge_index"][1, :]
    pg_full[i, :].scatter_add_(0, gen_bus, pg)
    qg_full[i, :].scatter_add_(0, gen_bus, qg)
    pd[i, :] = dataset[i]["load"]["x"][:, 0]
    qd[i, :] = dataset[i]["load"]["x"][:, 1]
    vm[i, :] = dataset[i]["bus"]["y"][:, 1]

pg = pg_full[:, spv_bus_mask]
qg = qg_full[:, spv_bus_mask]

opfdata = {"pd": pd, "qd": qd, "pg": pg, "qg": qg, "vm": vm}

In [ ]:
# Load OPFLearn Data
opflearn_data_path = "/Users/tamaragovindasamy/Desktop/gridfm-datakit/data/opflearn/pglib_opf_case118_ieee.csv"
df = pandas.read_csv(opflearn_data_path)
pd = df.filter(regex=r"^load\d+:pl$", axis=1).to_numpy()
qd = df.filter(regex=r"^load\d+:ql$", axis=1).to_numpy()
pg = torch.tensor(df.filter(regex=r"^gen\d+:pg$", axis=1).to_numpy())
qg = torch.tensor(df.filter(regex=r"^gen\d+:qg$", axis=1).to_numpy())
vm_df = df.filter(regex=r"^bus\d+:v_bus$", axis=1)


def safe_complex(val):
    if isinstance(val, str):
        val = val.replace(" + ", "+").replace(" - ", "-")  # fix spacing for conversion
        return complex(val)
    else:
        return np.nan


# Apply transformation
vm_complex = vm_df.applymap(safe_complex).to_numpy()
vm = np.abs(vm_complex)

# expand pg and qg
pg_full = torch.zeros((10000, 118), dtype=pg.dtype)
qg_full = torch.zeros((10000, 118), dtype=pg.dtype)
pg_full.scatter_add_(1, gen_bus_expanded, pg)
qg_full.scatter_add_(1, gen_bus_expanded, qg)
pg = pg_full[:, spv_bus_mask]
qg = qg_full[:, spv_bus_mask]

opflearn = {"pd": pd, "qd": qd, "pg": pg, "qg": qg, "vm": vm}

In [7]:
# Load datakit data
datakit_data_path = "/Users/tamaragovindasamy/Desktop/gridfm-datakit/data_out/case118_ieee_topology_no_gc/raw/pf_node.csv"
df_dk = pandas.read_csv(datakit_data_path)
df_dk = df_dk[(df_dk["scenario"] < 10000)]  # only 10000 samples
for col in ["Pd", "Qd", "Pg", "Qg"]:
    df_dk[col] = df_dk[col] / 100
df_dk = df_dk.rename(
    columns={"Pd": "pd", "Qd": "qd", "Pg": "pg", "Qg": "qg", "Vm": "vm", "Va": "va"}
)  # rename to match other data

# Need to group by generator bus and load bus indices first then select bus indices to plot

In [8]:
# Load PgLearn data
pglearn_data_path = (
    "/Users/tamaragovindasamy/Desktop/gridfm-datakit/data/pglearn/pglearn.csv"
)
df_pg = pandas.read_csv(pglearn_data_path)

df_pg = df_pg[(df_pg["scenario"] < 10000)]  # only 10000 samples
for col in ["Pd", "Qd", "Pg", "Qg"]:
    df_pg[col] = df_pg[col] / 100
df_pg = df_pg.rename(
    columns={"Pd": "pd", "Qd": "qd", "Pg": "pg", "Qg": "qg", "Vm": "vm", "Va": "va"}
)  # rename to match other data

In [ ]:
feature_cols = ["vm"]

# select 10  buses
sample_indices = [2, 14, 29, 45, 50, 56, 64, 77, 89, 108]


# Group data by bus
bus_groups_df_dk = df_dk.groupby("bus")
sorted_buses_df_dk = sorted(bus_groups_df_dk.groups.keys())

# sample_indices = random.sample(range(len(sorted_buses_df_dk)), 10)

bus_groups_df_pg = df_pg.groupby("bus")
sorted_buses_df_pg = sorted(bus_groups_df_pg.groups.keys())

for feature_name in feature_cols:
    fig, ax = plt.subplots(figsize=(15, 6))

    # other datasets
    bus_data_pfdelta = []  # pfdelta
    bus_data_opfdata = []  # opfdata
    bus_data_opflearn = []  # opflearn

    for i in sample_indices:  # Iterate over each bus
        bus_data_pfdelta.append(perturbpf[feature_name][:, i])
        bus_data_opfdata.append(opfdata[feature_name][:, i])
        bus_data_opflearn.append(opflearn[feature_name][:, i])

    bus_data_dk = [
        bus_groups_df_dk.get_group(bus)[feature_name].values for bus in sample_indices
    ]
    bus_data_pg = [
        bus_groups_df_pg.get_group(bus)[feature_name].values for bus in sample_indices
    ]

    datasets = [
        bus_data_dk,
        bus_data_opfdata,
        bus_data_pfdelta,
        bus_data_opflearn,
        bus_data_pg,
    ]

    # Create the plot
    x_pos = np.array(
        [np.arange(len(sample_indices)) + i * 0.2 for i in range(5)]
    ).T.flatten()

    # Plot
    colors = ["lightgreen", "pink", "thistle", "lightblue", "lightyellow"]

    for i, data in enumerate(datasets):
        bp = ax.boxplot(
            data,
            sym="",
            whis=[0, 100],
            widths=0.15,
            showfliers=False,
            showcaps=True,
            patch_artist=True,
            medianprops=dict(color="black", linewidth=1.5),
            tick_labels=list(datasets[0]),
            positions=[x_pos[i] + j * 1 for j in range(len(data))],
        )

        # parts = ax.violinplot(data, showmeans=True,
        #                    positions=[x_pos[i] + j * 1 for j in range(len(data))])

        # for pc in parts["bodies"]:
        #    #pc.set_facecolor("#DC91C4")
        #    pc.set_facecolor(colors[i])
        #    pc.set_alpha(0.7)

        for pc in bp["boxes"]:
            pc.set_facecolor(colors[i % 5])  # Cycles through colors
            pc.set_alpha(0.6)  # Transparency

    # Titles
    ax.set(
        title=f"{feature_name} Distribution Across Buses",
        ylabel=feature_name,
        xlabel="Bus Index",
    )
    # Remove the major x-axis tickmarks
    ax.tick_params(axis="x", bottom=False)
    xtick_positions = np.arange(len(sample_indices)) + 0.2
    ax.set_xticks(xtick_positions)

    # Positions of the x-axis labels
    ax.set_xlabel("Bus Index")
    ax.set_ylabel(feature_name)
    ax.set_xticklabels(
        [f"Bus {bus}" for bus in sample_indices],
        # rotation=45,
        ha="center",
    )
    ax.grid(True, alpha=0.3)

    # Add legend
    group_labels = ["GFM-Datakit", "OPF: n-1", "Pfdelta", "OPF Learn", "PgLearn"]
    legend_patches = [
        mpatches.Patch(color=colors[i], label=group_labels[i]) for i in range(5)
    ]
    ax.legend(handles=legend_patches, loc="best")
    plt.tight_layout()

    plt.show()

In [ ]:
feature_cols = ["pg"]

# select 10 buses
sample_indices = [7, 10, 14, 21, 24, 36, 38, 39, 41, 50]

# Group data by bus
bus_groups_df_dk = df_dk.groupby("bus")
sorted_buses_df_dk = sorted(bus_groups_df_dk.groups.keys())

# sample_indices = random.sample(range(len(sorted_buses_df_dk)), 10)

bus_groups_df_pg = df_pg.groupby("bus")
sorted_buses_df_pg = sorted(bus_groups_df_pg.groups.keys())

for feature_name in feature_cols:
    fig, ax = plt.subplots(figsize=(15, 6))

    # other datasets
    bus_data_pfdelta = []  # pfdelta
    bus_data_opfdata = []  # opfdata
    bus_data_opflearn = []  # opflearn

    for i in sample_indices:
        bus_data_pfdelta.append(perturbpf[feature_name][:, i])
        bus_data_opfdata.append(opfdata[feature_name][:, i])
        bus_data_opflearn.append(opflearn[feature_name][:, i])

    bus_data_dk = [
        bus_groups_df_dk.get_group(bus)[feature_name].values for bus in sample_indices
    ]
    bus_data_pg = [
        bus_groups_df_pg.get_group(bus)[feature_name].values for bus in sample_indices
    ]

    datasets = [
        bus_data_dk,
        bus_data_opfdata,
        bus_data_pfdelta,
        bus_data_opflearn,
        bus_data_pg,
    ]

    # Create the plot
    x_pos = np.array(
        [np.arange(len(sample_indices)) + i * 0.2 for i in range(5)]
    ).T.flatten()

    # Plot
    colors = ["lightgreen", "pink", "thistle", "lightblue", "lightyellow"]

    for i, data in enumerate(datasets):
        bp = ax.boxplot(
            data,
            sym="",
            whis=[0, 100],
            widths=0.15,
            showfliers=False,
            showcaps=True,
            patch_artist=True,
            medianprops=dict(color="black", linewidth=1.5),
            tick_labels=list(datasets[0]),
            positions=[x_pos[i] + j * 1 for j in range(len(data))],
        )

        # parts = ax.violinplot(data, showmeans=True,
        #                    positions=[x_pos[i] + j * 1 for j in range(len(data))])

        # for pc in parts["bodies"]:
        #    #pc.set_facecolor("#DC91C4")
        #    pc.set_facecolor(colors[i])
        #    pc.set_alpha(0.7)

        for pc in bp["boxes"]:
            pc.set_facecolor(colors[i % 5])  # Cycles through colors
            pc.set_alpha(0.6)  # Transparency

    # Titles
    ax.set(
        title=f"{feature_name} Distribution Across Buses",
        ylabel=feature_name,
        xlabel="Bus Index",
    )
    # Remove the major x-axis tickmarks
    ax.tick_params(axis="x", bottom=False)
    xtick_positions = np.arange(len(sample_indices)) + 0.2
    ax.set_xticks(xtick_positions)

    # Positions of the x-axis labels
    ax.set_xlabel("Bus Index")
    ax.set_ylabel(feature_name)
    ax.set_xticklabels(
        [f"Bus {bus}" for bus in sample_indices],
        # rotation=45,
        ha="center",
    )
    ax.grid(True, alpha=0.3)

    # Add legend
    group_labels = ["GFM-Datakit", "OPF: n-1", "Pfdelta", "OPF Learn", "PgLearn"]
    legend_patches = [
        mpatches.Patch(color=colors[i], label=group_labels[i]) for i in range(5)
    ]
    ax.legend(handles=legend_patches, loc="best")
    plt.tight_layout()
    plt.show()

In [ ]:
feature_cols = ["qg"]

# select 10 buses
sample_indices = [3, 7, 9, 12, 17, 30, 36, 37, 41, 45]

# Group data by bus
bus_groups_df_dk = df_dk.groupby("bus")
# sorted_buses_df_dk = sorted(bus_groups_df_dk.groups.keys())

# sample_indices = random.sample(range(len(sorted_buses_df_dk)), 10)

bus_groups_df_pg = df_pg.groupby("bus")
sorted_buses_df_pg = sorted(bus_groups_df_pg.groups.keys())

for feature_name in feature_cols:
    fig, ax = plt.subplots(figsize=(15, 6))

    # other datasets
    bus_data_pfdelta = []  # pfdelta
    bus_data_opfdata = []  # opfdata
    bus_data_opflearn = []  # opflearn

    for i in sample_indices:
        bus_data_pfdelta.append(perturbpf[feature_name][:, i])
        bus_data_opfdata.append(opfdata[feature_name][:, i])
        bus_data_opflearn.append(opflearn[feature_name][:, i])

    bus_data_dk = [
        bus_groups_df_dk.get_group(bus)[feature_name].values for bus in sample_indices
    ]
    bus_data_pg = [
        bus_groups_df_pg.get_group(bus)[feature_name].values for bus in sample_indices
    ]

    datasets = [
        bus_data_dk,
        bus_data_opfdata,
        bus_data_pfdelta,
        bus_data_opflearn,
        bus_data_pg,
    ]

    # Create the plot
    x_pos = np.array(
        [np.arange(len(sample_indices)) + i * 0.2 for i in range(5)]
    ).T.flatten()

    # Plot
    colors = ["lightgreen", "pink", "thistle", "lightblue", "lightyellow"]

    for i, data in enumerate(datasets):
        bp = ax.boxplot(
            data,
            sym="",
            whis=[0, 100],
            widths=0.15,
            showfliers=False,
            showcaps=True,
            patch_artist=True,
            medianprops=dict(color="black", linewidth=1.5),
            tick_labels=list(datasets[0]),
            positions=[x_pos[i] + j * 1 for j in range(len(data))],
        )

        # parts = ax.violinplot(data, showmeans=True,
        #                    positions=[x_pos[i] + j * 1 for j in range(len(data))])

        # for pc in parts["bodies"]:
        #    #pc.set_facecolor("#DC91C4")
        #    pc.set_facecolor(colors[i])
        #    pc.set_alpha(0.7)

        for pc in bp["boxes"]:
            pc.set_facecolor(colors[i % 5])  # Cycles through colors
            pc.set_alpha(0.6)  # Transparency

    # Titles
    ax.set(
        title=f"{feature_name} Distribution Across Buses",
        ylabel=feature_name,
        xlabel="Bus Index",
    )
    # Remove the major x-axis tickmarks
    ax.tick_params(axis="x", bottom=False)
    xtick_positions = np.arange(len(sample_indices)) + 0.2
    ax.set_xticks(xtick_positions)
    ax.set_xticklabels(
        [f"Bus {bus}" for bus in sample_indices],
        # rotation=45,
        ha="center",
    )
    # Positions of the x-axis labels
    ax.set_xlabel("Bus Index")
    ax.set_ylabel(feature_name)

    ax.grid(True, alpha=0.3)

    # Add legend
    group_labels = ["GFM-Datakit", "OPF: n-1", "Pfdelta", "OPF Learn", "PgLearn"]
    legend_patches = [
        mpatches.Patch(color=colors[i], label=group_labels[i]) for i in range(5)
    ]
    ax.legend(handles=legend_patches, loc="best")
    plt.tight_layout()
    plt.show()

In [ ]:
feature_cols = ["pd"]

# select 10 buses
sample_indices = [1, 6, 9, 26, 41, 43, 59, 75, 84, 93]

# Group data by bus
bus_groups_df_dk = df_dk.groupby("bus")
sorted_buses_df_dk = sorted(bus_groups_df_dk.groups.keys())

# sample_indices = random.sample(range(len(sorted_buses_df_dk)), 10)

bus_groups_df_pg = df_pg.groupby("bus")
sorted_buses_df_pg = sorted(bus_groups_df_pg.groups.keys())

for feature_name in feature_cols:
    fig, ax = plt.subplots(figsize=(15, 6))

    # other datasets
    bus_data_pfdelta = []  # pfdelta
    bus_data_opfdata = []  # opfdata
    bus_data_opflearn = []  # opflearn

    for i in sample_indices:
        bus_data_pfdelta.append(perturbpf[feature_name][:, i])
        bus_data_opfdata.append(opfdata[feature_name][:, i])
        bus_data_opflearn.append(opflearn[feature_name][:, i])

    bus_data_dk = [
        bus_groups_df_dk.get_group(bus)[feature_name].values for bus in sample_indices
    ]
    bus_data_pg = [
        bus_groups_df_pg.get_group(bus)[feature_name].values for bus in sample_indices
    ]

    datasets = [
        bus_data_dk,
        bus_data_opfdata,
        bus_data_pfdelta,
        bus_data_opflearn,
        bus_data_pg,
    ]

    # Create the plot
    x_pos = np.array(
        [np.arange(len(sample_indices)) + i * 0.2 for i in range(5)]
    ).T.flatten()

    # Plot
    colors = ["lightgreen", "pink", "thistle", "lightblue", "lightyellow"]

    for i, data in enumerate(datasets):
        bp = ax.boxplot(
            data,
            sym="",
            whis=[0, 100],
            widths=0.15,
            showfliers=False,
            showcaps=True,
            patch_artist=True,
            medianprops=dict(color="black", linewidth=1.5),
            tick_labels=list(datasets[0]),
            positions=[x_pos[i] + j * 1 for j in range(len(data))],
        )

        # parts = ax.violinplot(data, showmeans=True,
        #                    positions=[x_pos[i] + j * 1 for j in range(len(data))])

        # for pc in parts["bodies"]:
        #    #pc.set_facecolor("#DC91C4")
        #    pc.set_facecolor(colors[i])
        #    pc.set_alpha(0.7)

        for pc in bp["boxes"]:
            pc.set_facecolor(colors[i % 5])  # Cycles through colors properly
            pc.set_alpha(0.6)  # Set transparency

    # Titles
    ax.set(
        title=f"{feature_name} Distribution Across Buses",
        ylabel=feature_name,
        xlabel="Bus Index",
    )
    # Remove the major x-axis tickmarks
    ax.tick_params(axis="x", bottom=False)
    xtick_positions = np.arange(len(sample_indices)) + 0.2
    ax.set_xticks(xtick_positions)

    # Positions of the x-axis labels
    ax.set_xlabel("Bus Index")
    ax.set_ylabel(feature_name)
    ax.set_xticklabels(
        [f"Bus {bus}" for bus in sample_indices],
        # rotation=45,
        ha="center",
    )
    ax.grid(True, alpha=0.3)

    # Add legend
    group_labels = ["GFM-Datakit", "OPF: n-1", "Pfdelta", "OPF Learn", "PgLearn"]
    legend_patches = [
        mpatches.Patch(color=colors[i], label=group_labels[i]) for i in range(5)
    ]
    ax.legend(handles=legend_patches, loc="best")
    plt.tight_layout()
    plt.show()

In [ ]:
feature_cols = ["qd"]

# select 20 random buses
sample_indices = [7, 22, 33, 34, 40, 43, 65, 78, 92, 94]

# Group data by bus
bus_groups_df_dk = df_dk.groupby("bus")
sorted_buses_df_dk = sorted(bus_groups_df_dk.groups.keys())

# sample_indices = random.sample(range(len(sorted_buses_df_dk)), 10)

bus_groups_df_pg = df_pg.groupby("bus")
sorted_buses_df_pg = sorted(bus_groups_df_pg.groups.keys())

for feature_name in feature_cols:
    fig, ax = plt.subplots(figsize=(15, 6))

    # other datasets
    bus_data_pfdelta = []  # pfdelta
    bus_data_opfdata = []  # opfdata
    bus_data_opflearn = []  # opflearn

    for i in sample_indices:
        bus_data_pfdelta.append(perturbpf[feature_name][:, i])
        bus_data_opfdata.append(opfdata[feature_name][:, i])
        bus_data_opflearn.append(opflearn[feature_name][:, i])

    bus_data_dk = [  # datakit
        bus_groups_df_dk.get_group(bus)[feature_name].values for bus in sample_indices
    ]
    bus_data_pg = [  # pglearn
        bus_groups_df_pg.get_group(bus)[feature_name].values for bus in sample_indices
    ]

    datasets = [
        bus_data_dk,
        bus_data_opfdata,
        bus_data_pfdelta,
        bus_data_opflearn,
        bus_data_pg,
    ]

    # Create the plot
    # x_pos = np.array([np.arange(len(sample_indices)) + i * 0.2 for i in range(5)]).T.flatten()
    positions = np.array(
        [np.arange(len(sample_indices)) + i * 0.2 for i in range(3)]
    ).T.flatten()

    # Plot
    colors = ["lightgreen", "pink", "thistle", "lightblue", "lightyellow"]

    for i, data in enumerate(datasets):
        bp = ax.boxplot(
            data,
            sym="",
            whis=[0, 100],
            widths=0.15,
            showfliers=False,
            showcaps=True,
            patch_artist=True,
            medianprops=dict(color="black", linewidth=1.5),
            tick_labels=list(datasets[0]),
            positions=[x_pos[i] + j * 1 for j in range(len(data))],
        )

        # parts = ax.violinplot(data, showmeans=True,
        #                    positions=[x_pos[i] + j * 1 for j in range(len(data))])

        # for pc in parts["bodies"]:
        #    #pc.set_facecolor("#DC91C4")
        #    pc.set_facecolor(colors[i])
        #    pc.set_alpha(0.7)

        for pc in bp["boxes"]:
            pc.set_facecolor(colors[i % 5])  # Cycles through colors
            pc.set_alpha(0.6)  # Transparency

    # Titles
    ax.set(
        title=f"{feature_name} Distribution Across Buses",
        ylabel=feature_name,
        xlabel="Bus Index",
    )
    # Remove the major x-axis tickmarks
    ax.tick_params(axis="x", bottom=False)
    xtick_positions = np.arange(len(sample_indices)) + 0.2
    ax.set_xticks(xtick_positions)
    ax.set_xticklabels(
        [f"Bus {bus}" for bus in sample_indices],
        # rotation=45,
        ha="center",
    )

    # Positions of the x-axis labels
    ax.set_xlabel("Bus Index")
    ax.set_ylabel(feature_name)

    ax.grid(True, alpha=0.3)

    # Add legend
    group_labels = ["GFM-Datakit", "OPF: n-1", "Pfdelta", "OPF Learn", "PgLearn"]
    legend_patches = [
        mpatches.Patch(color=colors[i], label=group_labels[i]) for i in range(5)
    ]
    ax.legend(handles=legend_patches, loc="best")
    plt.tight_layout()
    plt.show()